# Лабораторная работа № 2
### Минимальная оценка - 3 балла
### Максимальная оценка - 5 балла
Цель работы: реализация основных алгоритмов поиска ассоциативных правил.

Для успешной сдачи лабораторной работы Вам необходимо знать, как работает реализуемый Вами алгоритм, предоставить заполненый исходный ноутбук и продемострировать процесс выполнения работы.

Данная лабораторная работа выполняется по вариантам. Номер варианта расчитывается по формуле:$((N+1) mod 2)+1$, где $N$ - ваш номер в списке группы.

## Задание 1 (3 балла)

### Задание 1.1 (общее для всех вариантов)

Описание датасета:
- каждая строка представляет собой одну транзакцию.
- каждый столбец представляет собой один товар (если значение столбца равно $1$, то товар присутствует в транзакции, иначе - отсутствует).

Пользуясь материалами лабораторной работы № 1:
1. загрузить данные из файла $marketbasket.csv$.
2. применить к ним обратное унитарное кодирование (reverse one-hot encoding).
3. удалить строки, в которых количество покупок меньше $5$.
4. сгенерировать свою подвыборку с количеством строк равным $20$ (сделать это можно, например, с помощью [DataFrame.sample](http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html)) ([DataFrame.sample](http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html) имеет параметр random_state, который Вам необходимо приравнять своему номеру в списке группы).

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("marketbasket.csv")

lst = []

for ind, row in df.iterrows():
    tmp = []
    for col in df.columns:
        if row[col] == 1:
            tmp.append(col)
    lst.append(tmp)

data = pd.DataFrame(lst)

In [2]:
data = data[data.count(axis=1) >= 5]
data.head()

,0,1,2,3,4,5,6,7,8,9,...,245,246,247,248,249,250,251,252,253,254
7,98pct. Fat Free Hamburger,Potato Chips,Sesame Oil,Ice Cream Sandwich,Frozen Cheese Pizza,Frozen Sausage Pizza,Deli Salad,Brown Sugar,Corn Chips,Merlot Wine,...,None,None,None,None,None,None,None,None,None,None
12,Sugar Cookies,Onions,Ice Cream,Instant Rice,Cantaloupe,Chocolate Chip Cookies,Corn Oil,Corn Chips,Paper Plates,Cabbage,...,None,None,None,None,None,None,None,None,None,None
13,98pct. Fat Free Hamburger,Onions,Strawberry Yogurt,Salsa Dip,Hot Dogs,Popcorn Salt,White Bread,Plastic Knives,Sunglasses,Hamburger Buns,...,None,None,None,None,None,None,None,None,None,None
22,98pct. Fat Free Hamburger,Sugar Cookies,Deli Ham,Potato Chips,Strawberry Yogurt,Mint Chocolate Bar,Buttered Popcorn,Waffles,Cream Soda,Apple Fruit Roll,...,None,None,None,None,None,None,None,None,None,None
24,98pct. Fat Free Hamburger,Sugar Cookies,Diet Soda,Salsa Dip,Buttered Popcorn,Mouthwash,Sugar,Smoked Turkey Sliced,Red Wine,Lettuce,...,None,None,None,None,None,None,None,None,None,None


In [3]:
data_sample = data.sample(n=20, random_state=5)
data_sample.head()

,0,1,2,3,4,5,6,7,8,9,...,245,246,247,248,249,250,251,252,253,254
970,Cottage Cheese,Sweat Potatoes,Waffles,Cream Soda,Shrimp Cocktail Sauce,Summer Sausage,Beef Soup,Hot Dogs,Raisins,Shampoo,...,None,None,None,None,None,None,None,None,None,None
916,98pct. Fat Free Hamburger,Sugar Cookies,Potato Chips,D Cell Batteries,Paper Towels,Salsa Dip,Chocolate Bar,Deodorant,Decaf Coffee,Screw Driver,...,None,None,None,None,None,None,None,None,None,None
445,Hair Conditioner,Cheese Crackers,Shrimp Cocktail Sauce,Summer Sausage,Plums,Strawberry Jam,Frozen Shrimp,Whole Corn,Fruit Drink,None,...,None,None,None,None,None,None,None,None,None,None
1162,Hair Conditioner,98pct. Fat Free Hamburger,Waffles,Dog Food,Mushrooms,Mushroom Pizza - Frozen,Deli Turkey,Beef Soup,Raisin Pudding,Napkins,...,None,None,None,None,None,None,None,None,None,None
372,Hair Conditioner,Standard coffee,98pct. Fat Free Hamburger,Strawberry Yogurt,Cheese Crackers,Waffles,Frozen Cheese Pizza,Manicotti,Liquid Laundry Detergent,Apple Cinnamon Waffles,...,None,None,None,None,None,None,None,None,None,None


### Задание 1.2
- **1 вариант**: пользуясь материалами лекции, реализовать алгоритм $Apriori$ и для своей подвыборки сгенерировать ассоциативные правила с уровнем уверенности не менее **50%** (уровень поддержки для часто встречающиеся подмножества объектов не менее **30%**).
- **2 вариант**: пользуясь материалами лекции, реализовать алгоритм $FP-Growth$ и для своей подвыборки найти часто встречающиеся подмножества объектов с уровнем поддержки не менее $2$.

In [46]:
from collections import Counter
import itertools
from IPython.display import display

def count(transactions, items):
    d = {}
    for item in items:
        for transaction in transactions:
            if type(item) != str:
                if set(i for i in item).issubset(transaction):
                    if item not in d:
                        d[item] = 1
                    else:
                        d[item] += 1
            else:
                if item in transaction:
                    if item not in d:
                        d[item] = 1
                    else:
                        d[item] += 1
    return d

def assotiative_rules(mvp, need_confidence):
    for elem in mvp:
        elem = set(elem)
        tmp = []
        for l in range(1, len(elem)):
            tmp+=list(itertools.combinations(elem, l))

        freq = count(ll, tmp)
        for j in tmp:
            confidence = 2/freq[j]
            if confidence > need_confidence:
                j = set(x for x in j)
                print(j, "=>", elem-j)

def apr(dataset, min_confidence=0.3, support=0.3, depth=5):
    bl = dataset.copy()
    l = dataset.copy()
    global len_dataset
    len_dataset = len(dataset)
    
    mvp = []
    for i in range(1, depth):
        l, lac = apriori(bl, l, support, i)
        if not l:
            break
        if lac:
            mvp += lac
    
    assotiative_rules(mvp, confidence)

def apriori(base_list, list_after_clipping, support, depth):
    # count будет считать количество вхождений элемента транзакции
    count = Counter()
    if depth == 1:
        pd_arr = pd.DataFrame(list_after_clipping)
        l1 = []
        for _, el in pd_arr.iterrows():
            l1.append(el.value_counts()) 
        for l in l1:
            count += Counter(l.to_dict())
    else:
        # comb_list это лист подстановок без повторений элементов транзакции
        comb_list = itertools.combinations(list_after_clipping, depth)
        for comb in list(comb_list):
            for row in base_list:
                tmp = True
                for elem in comb:
                    if elem not in row:
                        tmp = False
                if tmp:
                    if comb not in count.keys():
                        count[comb] = 1
                    else:
                        count[comb] += 1
    
    count_after_clipping = {}
    for elem, value in count.items():
        if value < support*len_dataset:
            continue
        count_after_clipping[elem] = value
    
    # Printing results of counting
    print("{} iteration".format(depth))
    df = pd.Series(count_after_clipping)
    display(df)
    
    list_after_clipping = list(count_after_clipping.keys())
    # array for next iteration
    afni = list_after_clipping if depth == 1 else list(set(itertools.chain(*list_after_clipping)))
    if depth>1:
        return afni, list_after_clipping
    return afni, None

ll = data_sample.values.tolist().copy()
mvp = apr(ll)

1 iteration


 Plain Bagels                   7
 Waffles                        7
 Oven Cleaner                   7
 Hamburger Buns                 9
 Sweet Relish                   9
 Hot Dogs                       6
 Chicken TV Dinner              6
 Eggs                          12
 Shrimp Cocktail Sauce          6
 Potato Chips                   6
 Pancake Mix                    6
 Orange Flavored Fruit Bars     6
 Salsa Dip                      6
 Plums                          7
 Bananas                        6
 98pct. Fat Free Hamburger     10
 Pepperoni Pizza - Frozen       7
 Cola                           6
 Sliced Chicken                 6
 White Bread                    9
 75 Watt Lightbulb              6
 French Fries                   6
 Onions                         6
 Oranges                        6
 Cantaloupe                     6
 Domestic Beer                  6
dtype: int64

2 iteration


 Plain Bagels                Sweet Relish                 6
 Oven Cleaner                Eggs                         6
 Hamburger Buns              Eggs                         6
                             98pct. Fat Free Hamburger    6
 Sweet Relish                Hot Dogs                     6
                             Eggs                         6
 Eggs                        98pct. Fat Free Hamburger    7
                             White Bread                  6
                             French Fries                 6
 Pancake Mix                 Salsa Dip                    6
 98pct. Fat Free Hamburger   White Bread                  6
dtype: int64

3 iteration


Series([], dtype: float64)

{' Plain Bagels'} => {' Sweet Relish'}
{' Oven Cleaner'} => {' Eggs'}
{' Hot Dogs'} => {' Sweet Relish'}
{' French Fries'} => {' Eggs'}
{' Pancake Mix'} => {' Salsa Dip'}
{' Salsa Dip'} => {' Pancake Mix'}


## Задание 2 (1 балл)
Реализовать алгоритм для другого варианта и продемонстрировать его работу на Вашей подвыборке.

In [ ]:
# ваш код

## Задание 3 (1 балл)
Для работы с большими данными и построения моделей можно использовать не только библиотеку sklearn, но и другие. Один из примеров - **[Weka](http://www.machinelearning.ru/wiki/index.php?title=WEKA)**.

Для выполнения этого задания Вам необходимо:
1. Установить **Weka** на свой компьютер (скачать можно вот [тут](https://www.cs.waikato.ac.nz/ml/weka/downloading.html)).
2. Запустить графический интерфейс программы.
3. Загрузить туда файл $zoo.csv$ и предобработать его (путём удаления столбца, описывающего количество лап у животных (legs), с помощью фильтра удаления (**remove filter**)).
4. Запустить алгоритм $Apriori$ и продемонстрировать его работу на этом наборе данных (параметры выбрать самостоятельно).

In [ ]:
# задание 3 выполняется в программе Weka, код на Python писать не нужно !!!